In [1]:
from utils.file_utils import get_config, get_npy_files
from dataset.occ_flow_utils import GridMap
import typing
import numpy as np
import pandas as pd
import easydict as edict
import json
import torch
from torch.utils.data import DataLoader, Dataset
import time

In [2]:
class I24Dataset(Dataset):
    def __init__(self, config):
        self.config = config
        self.grid_map = GridMap(config)
        self.data_files = get_npy_files(config.dataset.processed_data)
        
    def add_occ_flow(self, feature_dic):
        occ_map, flow_map = self.grid_map.get_map_flow(feature_dic)
        feature_dic['occupancy_map'] = occ_map
        feature_dic['flow_map'] = flow_map
        return feature_dic 
    
    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        data_dic = np.load(self.data_files[idx], allow_pickle=True).item()
        
        # Create the feature dictionary to save
        his_len = self.config.dataset.his_len
        pred_len = self.config.dataset.pred_len
        feature_dic = typing.DefaultDict(dict)
        for dic_k, dic in data_dic.items():
            dic = self.add_occ_flow(dic)
            for k, v in dic.items():
                if k in ['timestamp', 'x_position', 'y_position', 'x_velocity', 'y_velocity', 'yaw_angle']:
                    # (Num of Agents, Timestamp)

                    feature_dic[dic_k + '/state/his/' + k] = v[:, :his_len]
                    feature_dic[dic_k + '/state/pred/' + k] = v[: his_len: his_len + pred_len]
                elif k in ['occupancy_map', 'flow_map']:
                    # (Timestamp, H, W) -> Occ (Timestamp, H, W, 2) -> Flow
                    feature_dic[dic_k + '/state/his/' + k] = v[:his_len,...]
                    feature_dic[dic_k + '/state/pred/' + k] = v[his_len: his_len + pred_len,...]
                    # pred_v = v[his_len: his_len + pred_len,...]
                    # pred_v = pred_v.reshape(-1, pred_len//10, *pred_v.shape[1:]).sum(axis=0)
                    # print(f'{k}, pred {pred_v.shape}')
                else:
                    feature_dic[dic_k + '/meta/' + k] = v
            
        return feature_dic

In [3]:
import torch

def merge_batch_by_padding_2nd_dim(tensor_list, return_pad_mask=False):
    # Determine the dimensions of the tensors
    tensor_shape_len = len(tensor_list[0].shape)
    if tensor_shape_len not in [2]:
        return torch.stack(tensor_list, dim=0)

    # Flag for determining if we need to adjust the dimensions back
    only_2d_tensor = tensor_shape_len == 2

    # If the input tensors are 2D, add an extra dimension to make it compatible with 3D tensors
    if only_2d_tensor:
        tensor_list = [x.unsqueeze(dim=-1) for x in tensor_list]  # Convert to (batch, length, 1, 1)
        
    
    # Calculate the maximum length along the second dimension
    maxt_feat0 = max([x.shape[0] for x in tensor_list])
    
    # Retrieve the shape attributes after adjustment
    _, num_feat1, num_feat2 = tensor_list[0].shape

    # Initialize lists to store padded tensors and mask tensors
    ret_tensor_list = []
    ret_mask_list = []

    # Pad tensors and generate masks
    for k in range(len(tensor_list)):
        cur_tensor = tensor_list[k]

        # Create a new tensor with the maximum shape and copy the current tensor into it
        new_tensor = cur_tensor.new_zeros(maxt_feat0, num_feat1, num_feat2)
        new_tensor[:cur_tensor.shape[0], :, :] = cur_tensor
        # new_tensor = new_tensor[None,]
        ret_tensor_list.append(new_tensor)
        # Create a mask tensor indicating the padded regions
        new_mask_tensor = cur_tensor.new_zeros(maxt_feat0)
        new_mask_tensor[:cur_tensor.shape[0]] = 1
        ret_mask_list.append(new_mask_tensor.bool())
        
    # Concatenate the padded tensors and masks along the batch dimension
    ret_tensor = torch.stack(ret_tensor_list, dim=0)  # (num_stacked_samples, maxt_feat0, num_feat1, num_feat2)
    ret_mask = torch.stack(ret_mask_list, dim=0)
    # print(ret_tensor.shape)
    # If the input was originally a 2D tensor, squeeze back to the original number of dimensions
    if only_2d_tensor:
        
        ret_tensor = ret_tensor.squeeze(dim=-1).squeeze(dim=-1)  # Remove last two dimensions

    if return_pad_mask:
        return ret_tensor, ret_mask
    else:
        return ret_tensor


def process_batch(batch_list):
    key_to_list = {}
    batch_size = len(batch_list)
    meta_scalar = [
                    'num_vehicles',
                    # '_id', 
                #    'scene_id', 
                   'start_pos', 
                   'start_time'
                   ]
    meta_array = ['length', 
                  'width', 
                #   'height', 
                  'class', 
                  'direction']
    state = ['timestamp', 'x_position', 'y_position', 'x_velocity', 'y_velocity', 'yaw_angle', 'occupancy_map', 'flow_map']
    state_keys = []
    meta_scalar_keys = []
    meta_array_keys = []
    for scene_key in ['prv', 'cur', 'nxt']:
        for key in meta_scalar:
            key = f'{scene_key}/meta/{key}'
            meta_scalar_keys.append(key)
        for key in meta_array:
            key = f'{scene_key}/meta/{key}'
            meta_array_keys.append(key)
        for key in state:
            his_key = f'{scene_key}/state/his/{key}'
            pred_key = f'{scene_key}/state/pred/{key}'
            state_keys.extend([his_key, pred_key])
            
    for key in meta_scalar_keys:
        key_to_list[key] = [batch_list[bs_idx][key] for bs_idx in range(batch_size)]
    for key in meta_array_keys:
        key_to_list[key] = [batch_list[bs_idx][key] for bs_idx in range(batch_size)]
    for key in state_keys:
        key_to_list[key] = [batch_list[bs_idx][key] for bs_idx in range(batch_size)]
    
    
    input_dict = {}
    for key, val_list in key_to_list.items():
        if key in state_keys:
            
            val_list = [torch.from_numpy(x) for x in val_list]
            input_dict[key] = merge_batch_by_padding_2nd_dim(val_list)
            
        elif key in meta_scalar_keys:
            # chec if value list are zero-dimensional arrays
            if not isinstance(val_list[0], np.ndarray):
                
                input_dict[key] = torch.tensor(val_list)
            else:   
                input_dict[key] = np.concatenate(val_list, axis=0)
        else:
            
                val_list = [torch.from_numpy(x) for x in val_list]
                input_dict[key] = torch.cat(val_list, dim=0)
    return input_dict

def collate_fn(batch_list):
    batch_size = len(batch_list)
    
    feature_dic = process_batch([_ for _ in batch_list])
    # cur_num_vehicles = [len(batch_list[bs_idx]['cur/meta/num_vehicles']) for bs_idx in range(batch_size)]
    batch_dict = {'batch_size': batch_size,
                  'feature_dic': feature_dic}
                  
    
    return batch_dict

config = get_config()




dataset = I24Dataset(config)


In [7]:
import time
dataset = I24Dataset(config)
dataloader = DataLoader(dataset, batch_size=20, shuffle=False, collate_fn=collate_fn, num_workers=20)
prv_time = time.time()
for data in dataloader:
    
    print(data['batch_size'])
    print(f'time {time.time() - prv_time}')
    prv_time = time.time()

20
time 24.422866582870483
16
time 0.6264431476593018


{'batch_size': 20,
 'feature_dic': {'prv/meta/num_vehicles': tensor([ 0,  3,  2,  0,  5,  0,  0,  2, 10,  0,  5, 12,  2,  2,  5,  0,  3, 10,
          14,  6]),
  'prv/meta/start_pos': tensor([309973.4688, 314223.4688, 313973.4688, 309723.4688, 317473.4688,
          310223.4688, 311223.4688, 315973.4688, 314723.4688, 310973.4688,
          313223.4688, 317973.4688, 311973.4688, 315473.4688, 313473.4688,
          310473.4688, 315723.4688, 314973.4688, 318223.4688, 315223.4688],
         dtype=torch.float64),
  'prv/meta/start_time': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'cur/meta/num_vehicles': tensor([ 0,  3,  2,  0,  5,  0,  0,  2, 10,  0,  5, 12,  2,  2,  5,  0,  3, 10,
          14,  6]),
  'cur/meta/start_pos': tensor([310223.4688, 314473.4688, 314223.4688, 309973.4688, 317723.4688,
          310473.4688, 311473.4688, 316223.4688, 314973.4688, 311223.4688,
          313473.4688, 318223.4688, 312223.4688, 315723.4688, 313723.4688,
          310723

In [ ]:

for i in range(100) :
    if occ_map[i].sum() > 0:
        print(i)
        break
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import datetime

# Example data (replace with your actual data)
flow_data = flow.swapaxes(-3, -2) # Shape: (time, x, y, [dx, dy])
test_map = map.swapaxes(-1, -2)  # Occupancy map (time, x, y)
start_time = datetime.datetime.fromtimestamp(1638320400)  # Example UNIX timestamp
start_pos = 5000  # Example starting position (in feet)

# Initialize the plot
fig, ax = plt.subplots(figsize=(12, 6))  # Adjust size for better layout

# Create the meshgrid for the quiver plot
x = np.arange(0, flow_data.shape[2])  # X-axis points (256)
y = np.arange(0, flow_data.shape[1])  # Y-axis points (128)
X, Y = np.meshgrid(x, y)

# Initialize the occupancy map with the first frame's data
img = ax.imshow(test_map[0], cmap='viridis', interpolation='nearest', alpha=0.6)

# Initialize the quiver plot with the first frame's data
frame_0_flow = flow_data[0]
quiver = ax.quiver(X, Y, frame_0_flow[..., 0], frame_0_flow[..., 1], 
                   angles='xy', scale_units='xy', scale=1, color='white')

# Set plot title and labels
ax.set_title("Flow Field and Occupancy Map", fontsize=16, fontweight='bold')
ax.set_xlabel("X-axis", fontsize=12)
ax.set_ylabel("Y-axis", fontsize=12)

# Add a timestamp annotation
timestamp = ax.text(
    0.02, 0.95, f"Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}", 
    transform=ax.transAxes, fontsize=10, color='white',
    bbox=dict(facecolor='black', alpha=0.5), ha='left', va='center'
)

# Animation update function
def update(frame):
    """Update both the quiver plot and the occupancy map for each frame."""
    # Update occupancy map
    img.set_data(test_map[frame])
    
    # Update quiver plot
    quiver.set_UVC(flow_data[frame, ..., 0], flow_data[frame, ..., 1])
    
    # Update timestamp
    current_time = start_time + datetime.timedelta(seconds=frame / 20)  # Assuming 20 FPS
    timestamp.set_text(f"Time: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    return [img, quiver, timestamp]

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=len(flow_data), interval=50, blit=True)

# Save the animation as a video (optional)
ani.save('flow_occupancy_animation_04.mp4', writer='ffmpeg', fps=20, dpi=150)

# Display the animation
plt.show()